In [33]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from configs import get_config

DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(DEVICE)

mps


Upload the labels.csv and processed_counts.csv files to colab or your local workspace.

This data associates a cell barcode, such as "AAAGCCTGGCTAAC-1", to a certain cell type label, such as "CD14+ Monocyte". For each cell barcode, there are also log RNA seq counts of 765 different genes, such as HES4.

label.csv stores the association between a cell barcode and a cell type label.

processed_counts.csv stores the normalized log read counts for each cell, where each row represents a single cell, and each column represents a gene.

In [9]:
labels_pd = pd.read_csv("labels.csv")
counts_pd = pd.read_csv("processed_counts.csv")

In [84]:
labels_pd.shape, counts_pd.shape

((700, 2), (700, 766))

In [67]:
labels = labels_pd['index'].to_numpy().astype(str)
labels[[0,1,2]]

array(['AAAGCCTGGCTAAC-1', 'AAATTCGATGCACA-1', 'AACACGTGGTCTTT-1'],
      dtype='<U16')

In [16]:
id2cell = {i:label for i, label in enumerate(labels_pd['bulk_labels'].unique())}
cell2id = {label:i for i, label in id2cell.items()}

({'CD14+ Monocyte': 0,
  'Dendritic': 1,
  'CD56+ NK': 2,
  'CD4+/CD25 T Reg': 3,
  'CD19+ B': 4,
  'CD8+ Cytotoxic T': 5,
  'CD4+/CD45RO+ Memory': 6,
  'CD8+/CD45RA+ Naive Cytotoxic': 7,
  'CD4+/CD45RA+/CD25- Naive T': 8,
  'CD34+': 9},
 {0: 'CD14+ Monocyte',
  1: 'Dendritic',
  2: 'CD56+ NK',
  3: 'CD4+/CD25 T Reg',
  4: 'CD19+ B',
  5: 'CD8+ Cytotoxic T',
  6: 'CD4+/CD45RO+ Memory',
  7: 'CD8+/CD45RA+ Naive Cytotoxic',
  8: 'CD4+/CD45RA+/CD25- Naive T',
  9: 'CD34+'})

In [112]:
labels = ['CD14+ Monocyte','Dendritic','CD56+ NK','CD4+/CD25 T Reg','CD19+ B','CD8+ Cytotoxic T','CD4+/CD45RO+ Memory','CD8+/CD45RA+ Naive Cytotoxic','CD4+/CD45RA+/CD25- Naive T','CD34+']
id2cell = {i:label for i, label in enumerate(labels)}
cell2id = {label:i for i, label in id2cell.items()}
id2cell, cell2id

({0: 'CD14+ Monocyte',
  1: 'Dendritic',
  2: 'CD56+ NK',
  3: 'CD4+/CD25 T Reg',
  4: 'CD19+ B',
  5: 'CD8+ Cytotoxic T',
  6: 'CD4+/CD45RO+ Memory',
  7: 'CD8+/CD45RA+ Naive Cytotoxic',
  8: 'CD4+/CD45RA+/CD25- Naive T',
  9: 'CD34+'},
 {'CD14+ Monocyte': 0,
  'Dendritic': 1,
  'CD56+ NK': 2,
  'CD4+/CD25 T Reg': 3,
  'CD19+ B': 4,
  'CD8+ Cytotoxic T': 5,
  'CD4+/CD45RO+ Memory': 6,
  'CD8+/CD45RA+ Naive Cytotoxic': 7,
  'CD4+/CD45RA+/CD25- Naive T': 8,
  'CD34+': 9})

In [124]:
labels_pd['index'].to_numpy().astype(str)[:20]

array(['AAAGCCTGGCTAAC-1', 'AAATTCGATGCACA-1', 'AACACGTGGTCTTT-1',
       'AAGTGCACGTGCTA-1', 'ACACGAACGGAGTG-1', 'ACAGTTCTTAGCCA-1',
       'ACATTCTGACTACG-1', 'ACCCTCGAGTGAGG-1', 'ACTGGCCTTTTCGT-1',
       'ACTTGGGAACCAGT-1', 'AGAAAGTGTGAACC-1', 'AGATATTGACCACA-1',
       'AGTAAGGATTTACC-1', 'AGTTAAACAAACAG-1', 'ATAACAACCTCTAT-1',
       'ATGGACACAAGTGA-1', 'ATGGACACTCGTTT-1', 'ATGGGTACCTGGTA-1',
       'ATGTAAACTTTCGT-1', 'ATTAACGATACGAC-1'], dtype='<U16')

In [153]:
numpy_counts = counts_pd.iloc[:,1:].to_numpy().astype(np.float32)
torch.from_numpy(numpy_counts).size()

torch.Size([700, 765])

Shuffle your data. Make sure your labels and the counts are shuffled together.

Split into train and test sets (80:20 split)

In [146]:
"""
I might want to separate out the labels/indicies from the counts into two separate datasets, stacking them at the end using torch.utils.data.StackDataset
based on if it is training or not. I don't know if returning the labels and 
"""

class CellDataset(Dataset):
    def __init__(self, counts_csv_file:str, labels_csv_file:str, mode='train', test_size=0.2) -> tuple[torch.tensor, torch.tensor, np.ndarray]:
        assert mode in ['train', 'test'], f'mode needs to be either train or test, but it\'s {mode}'

        self.labels = ['CD14+ Monocyte','Dendritic','CD56+ NK','CD4+/CD25 T Reg','CD19+ B','CD8+ Cytotoxic T','CD4+/CD45RO+ Memory','CD8+/CD45RA+ Naive Cytotoxic','CD4+/CD45RA+/CD25- Naive T','CD34+']
        self.id2cell = {i:label for i, label in enumerate(self.labels)}
        self.cell2id = {label:i for i, label in self.id2cell.items()}
        
        counts_pd = pd.read_csv(counts_csv_file)
        numpy_counts = counts_pd.iloc[:,1:].to_numpy().astype(np.float32) # iloc[:, 1:] gets rid of the index column, leaving only normalized log count data

        labels_pd = pd.read_csv(labels_csv_file)
        id_labels = labels_pd['bulk_labels'].map(cell2id).to_numpy().astype(np.uint8)

        partition = int(len(labels_pd) * (1 - test_size))
    
        if mode == 'train':
            self.counts = torch.from_numpy(numpy_counts[:partition, :])
            self.labels = torch.from_numpy(id_labels[:partition])
            self.indicies = labels_pd['index'].to_numpy().astype(str)[:partition]
        else:
            self.counts = torch.from_numpy(numpy_counts[partition:, :])
            self.labels = torch.from_numpy(id_labels[partition:])
            self.indicies = labels_pd['index'].to_numpy().astype(str)[partition:]
            

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        log_counts = self.counts[idx]
        label = self.labels[idx]
        index = self.indicies[idx]
        return log_counts, label, index

    # def __getitems__(self, idxs):
    #     counts = self.counts[idxs]
    #     labels = self.labels[idxs]
    #     indicies = self.indicies[idxs]
    #     return counts, labels, indicies

In [152]:
train_data = CellDataset("processed_counts.csv", "labels.csv", mode='train')
test_data = CellDataset("processed_counts.csv", "labels.csv", mode='test')
train_data.labels.shape, test_data.labels.shape

(torch.Size([560]), torch.Size([140]))

Create a fully connected neural network for your autoencoder. Your latent space can be of any size less than or equal to 64. Too large may result in a poor visualization, and too small may result in high loss. 32 is a good starting point.

Consider using more than 1 hidden layer, and a sparcity constraint (l1 regularization).

Have an encoder model which is a model of only the layers for the encoding.

In [35]:
MODEL_CONFIG = get_config("autoencoder_model.config")

In [252]:
input_size, num_hidden_layers, output_size = 32, 3, 766
jump = int(np.abs(np.ceil((input_size - output_size)/(num_hidden_layers))))
print(f"Jump: {np.abs((input_size - output_size) / (num_hidden_layers))}")

Jump: 244.66666666666666


In [253]:
size = input_size
layer_sizes = [input_size]
for _ in range(num_hidden_layers - 1):
    if input_size > output_size:
        size -= jump
    elif input_size < output_size:
        size += jump
    layer_sizes.append(size)
else:
    layer_sizes = [output_size for _ in range(num_hidden_layers - 1)]

# layer_output_sizes.append(output_size)
layer_sizes

[766, 766]

In [250]:
[layer_sizes[i: i + 2] for i in range(len(layer_sizes) - 2 + 1)]

[[32, 276], [276, 520]]

In [157]:
class NodeLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(NodeLayer, self).__init__(
            nn.Linear(input_size, output_size),
            nn.ReLU()
        )
        
class Encoder(nn.Module):
    def __init__(self, input_size, num_hidden_layers, output_size):
        super(Encoder, self).__init__()

        hidden_dims = 
        
        self.hidden_layers = nn.Sequential(*[NodeLayer(input_size, connection_size) for hidden])
        self.out_layer = nn.Linear(hidden_dims[-1], output_size)

    def forward(self, x):
        x = self.hidden_layers(x)
        return self.out_layer(x)

class Decoder(nn.Module):
    def __init__(self, input_size, num_hidden_layers, output_size):
        super(Decoder, self).__init__()

        hidden_dims = 
        
        self.hidden_layers = nn.Sequential(*[NodeLayer(input_size, connection_size) for connection_size])
        self.out_layer = nn.Linear(hidden_dims[-1], output_size)

    def forward(self, x):
        x = self.hidden_layers(x)
        return self.out_layer(x)

class PBMCAutoEncoder(nn.Module):
    def __init__(self):
        super(PBMCAutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


SyntaxError: '[' was never closed (683309363.py, line 10)

Train your autoencoding using MSE loss.

Finally, identify the parameters which don't overfit, and use the same model architecture and train on all of the data together.

With a latent space size of 32, aim for 0.9 MSE loss on your test set, 0.95 with regularization. You will not be graded strictly on a loss cutoff.

In [47]:
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm, trange

In [156]:
def train(model, train_data:Dataset, test_data:Dataset, regularization:bool):
    # torch.cuda.empty_cache()
    model = model.to(DEVICE)

    train_loader = DataLoader(train_data, batch_size=MODEL_CONFIG['hyperparameters']['BATCH_SIZE'], shuffle=True)
    test_loader = DataLoader(test_data, batch_size=MODEL_CONFIG['hyperparameters']['BATCH_SIZE'], shuffle=False)

    # best_val_loss = float('inf')
    # best_model = None
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }

    mse = nn.MSELoss()
    if regularization == True:
        l1 = nn.L1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=MODEL_CONFIG['hyperparameters']['LR'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                           factor=MODEL_CONFIG['hyperparameters']['LR_REDUCE_RATE'], 
                                                           patience=MODEL_CONFIG['hyperparameters']['PATIENCE'])

    epochs_pbar = trange(MODEL_CONFIG['hyperparameters']['EPOCHS'], desc='Epochs')
    for epoch in epochs_pbar:
        # Training
        model.train()
        epoch_train_loss = 0.0
        train_preds, train_targets = [], []
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)

            logits = model(data)
            if regularization == True:
                loss = mse(logits, labels) + l1(logits, labels)
            else:
                loss = mse(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item() * data.size(0)

            _, predicted = torch.max(logits.data, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        epoch_train_loss /= len(training_data)
        epoch_train_acc = accuracy_score(train_targets, train_preds)

        # Testing
        model.eval()
        epoch_val_loss = 0.0
        val_preds, val_targets = [], []

        with torch.no_grad():
            for imgs, labels in test_loader:
                imgs, labels = imgs.to(device), labels.to(device)

                logits = model(imgs)
                loss = loss_fn(logits, labels)

                epoch_val_loss += loss.item() * imgs.size(0)

                _, predicted = torch.max(logits.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        epoch_val_loss /= len(testing_data)
        epoch_val_acc = accuracy_score(val_targets, val_preds)
        
        print(f'Epoch {epoch+1} -- Val loss: {epoch_val_loss:.4f} Train loss: {epoch_train_loss:.4f}  '
              f'Val acc: {epoch_val_acc:.4f} Train acc: {epoch_train_acc:.4f}')

        scheduler.step(epoch_val_loss)

        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)

        # if epoch_val_loss > best_val_loss:
        #     best_val_loss = epoch_val_loss
        #     best_model = deepcopy(model.state_dict())

    plot_history(history)
            
    return {
        'model': model,
        'history': history,
        'best_val_loss': best_val_loss,
        'optimizer_state': optimizer.state_dict()
    }

In [154]:
def plot_history(history):# Plot training history
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.tight_layout()
    plt.show()

    model.load_state_dict(best_model.state_dict())

In [100]:
from time import sleep
from random import random
epochs_pbar = trange(4, desc='Epochs')
for epoch in epochs_pbar:
    for j in range(100):
        sleep(0.01)
    for k in range(100):
        sleep(0.01)

    print(f'Epoch {epoch+1} -- Val loss: {0.01:.4f} Train loss: {0.01:.4f}  Val acc: {0.01:.4f} Test acc: {0.01:.4f}')

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1 -- Val loss: 0.0100 Train loss: 0.0100  Val acc: 0.0100 Test acc: 0.0100
Epoch 2 -- Val loss: 0.0100 Train loss: 0.0100  Val acc: 0.0100 Test acc: 0.0100
Epoch 3 -- Val loss: 0.0100 Train loss: 0.0100  Val acc: 0.0100 Test acc: 0.0100
Epoch 4 -- Val loss: 0.0100 Train loss: 0.0100  Val acc: 0.0100 Test acc: 0.0100


In [75]:
with trange(4) as t:
    for i in t:
        t.set_description(f'Epoch {i+1}')
        with trange(100, desc='Train') as tr:
            for j in tr:
                tr.set_postfix(test_loss=random())
                sleep(0.01)
        for k in trange(100, desc='Test'):
            sleep(0.01)

  0%|          | 0/4 [00:00<?, ?it/s]

Train:   0%|          | 0/100 [00:00<?, ?it/s]

Test:   0%|          | 0/100 [00:00<?, ?it/s]

Train:   0%|          | 0/100 [00:00<?, ?it/s]

Test:   0%|          | 0/100 [00:00<?, ?it/s]

Train:   0%|          | 0/100 [00:00<?, ?it/s]

Test:   0%|          | 0/100 [00:00<?, ?it/s]

Train:   0%|          | 0/100 [00:00<?, ?it/s]

Test:   0%|          | 0/100 [00:00<?, ?it/s]

Use PCA and t-SNE on the dataset.

Then use PCA on the latent space representation of the dataset.

Plot all of these.

Compare the results of PCA, t-SNE, and your autoencoder as ways to visualize the data.